<a href="https://colab.research.google.com/github/wojiushilr/ML_hw/blob/master/hw4_LSTM_semi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
! pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 4.0MB 27.8MB/s 
  Found existing installation: torch 1.5.0
    Uninstalling torch-1.5.0:
      Successfully uninstalled torch-1.5.0
  Found existing installation: torchvision 0.6.0
    Uninstalling torchvision-0.6.0:
      Successfully uninstalled torchvision-0.6.0


In [ ]:
! pip list

### load the pre-train w2v model

In [4]:
!cp "/content/drive/My Drive/kaggle/hw4/ml2020spring-hw4.zip" /content
!cp "/content/drive/My Drive/kaggle/hw4/w2v_all.model" /content
! cp "/content/drive/My Drive/kaggle/hw4/w2v_all.model.wv.vectors.npy" /content
! cp "/content/drive/My Drive/kaggle/hw4/w2v_all.model.trainables.syn1neg.npy" /content

### load the dataset

In [5]:
!ls
!unzip ml2020spring-hw4.zip

drive		      sample_data    w2v_all.model.trainables.syn1neg.npy
ml2020spring-hw4.zip  w2v_all.model  w2v_all.model.wv.vectors.npy
Archive:  ml2020spring-hw4.zip
  inflating: testing_data.txt        
  inflating: training_label.txt      
  inflating: training_nolabel.txt    


In [6]:
!ls

drive		      training_nolabel.txt
ml2020spring-hw4.zip  w2v_all.model
sample_data	      w2v_all.model.trainables.syn1neg.npy
testing_data.txt      w2v_all.model.wv.vectors.npy
training_label.txt


In [7]:
# 设置后可以过滤一些无用的warning
import warnings
warnings.filterwarnings('ignore')

### 這個 block 用來先定義一些等等常用到的函式


In [8]:
def load_training_data(path='training_label.txt'):
    if "training_label" in path:
        with open(path, 'r') as f:
            lines = f.readlines()
            lines = [line.strip('\n').split(' ') for line in lines]
            print(lines[0])
        x = [line[2:] for line in lines]
        y = [line[0] for line in lines]
        # print(x[0])
        # print(y[0])
        return x, y
    else:
        with open(path, 'r') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
        return x

def load_testing_data(path='testing_data'):
    # 把 testing 時需要的 data 讀進來
    with open(path, 'r') as f:
        lines = f.readlines()
        X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]
        X = [sen.split(' ') for sen in X]
        print(X[0])
    return X

def evaluation(outputs, labels):
    # outputs => probability (float)
    # labels => labels
    outputs[outputs>=0.5] = 1 # 大於等於 0.5 為正面
    outputs[outputs<0.5] = 0 # 小於 0.5 為負面
    correct = torch.sum(torch.eq(outputs, labels)).item()
    return correct

### 這個 block 是用來訓練 word to vector 的 word embedding
### 预计embedding耗时10min

In [ ]:
# '''
# 读取数据
# w2v.py
# 這個 block 是用來訓練 word to vector 的 word embedding
# '''
# import os
# import numpy as np
# import pandas as pd
# import argparse
# from gensim.models import Word2Vec
# def train_word2vec(x):
#     # 訓練 word to vector 的 word embedding
#     model = Word2Vec(x, size=250, window=5, min_count=5, workers=12, iter=10, sg=1)
#     return model
# if __name__ == "__main__":
#     print("loading training data ...")
#     train_x, y = load_training_data('training_label.txt')
#     train_x_no_label = load_training_data('training_nolabel.txt')

#     print("loading testing data ...")
#     test_x = load_testing_data('testing_data.txt')

#     # model = train_word2vec(train_x + train_x_no_label + test_x)
#     model = train_word2vec(train_x + test_x)

#     print("saving model ...")
#     # model.save(os.path.join(path_prefix, 'model/w2v_all.model'))
#     model.save(os.path.join('w2v_all.model'))

### 数据处理
### NLP类似问题可以重复使用

In [9]:
class Preprocess():
    def __init__(self, sen_len, w2v_path="./w2v.model"):
        self.w2v_path = w2v_path   # word2vec的存储路径
        self.sen_len = sen_len    # 句子的固定长度
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
    def get_w2v_model(self):
        # 把之前訓練好的 word to vec 模型讀進來
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
    def add_embedding(self, word):
        # 把 word 加進 embedding，並賦予他一個隨機生成的 representation vector
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector) #  均匀分布
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0) # 指定维度为0，所以按行拼接

    def make_embedding(self, load=True):
        print("Get embedding ...")
        # 取得訓練好的 Word2vec word embedding
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
        # 製作一個 word2idx 的 dictionary
        # 製作一個 idx2word 的 list
        # 製作一個 word2vector 的 list
        for i, word in enumerate(self.embedding.wv.vocab):
            print('get words #{}'.format(i + 1), end='\r')
            # e.g. self.word2index['he'] = 1
            # e.g. self.index2word[1] = 'he'
            # e.g. self.vectors[1] = 'he' vector
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        # 將 "<PAD>" 跟 "<UNK>" 加進 embedding 裡面
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix

    def pad_sequence(self, sentence):
        # 将每个句子变成一样的长度，即 sen_len 的长度
        if len(sentence) > self.sen_len:
        # 如果句子长度大于 sen_len 的长度，就截断
            sentence = sentence[:self.sen_len]
        else:
        # 如果句子长度小于 sen_len 的长度，就补上 <PAD> 符号，缺多少个单词就补多少个 <PAD> 
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len
        return sentence
    def sentence_word2idx(self, sentences):
        sentence_list = []
        for i, sen in enumerate(sentences):
            print('sentence count #{}'.format(i+1), end='\r')
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                # 没有出现过的单词就用 <UNK> 表示
                    sentence_idx.append(self.word2idx["<UNK>"])
            # 将每个句子变成一样的长度
            sentence_idx = self.pad_sequence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)
    def labels_to_tensor(self, y):
        # 把 labels 轉成 tensor
        y = [int(label) for label in y]
        return torch.LongTensor(y)
    def get_pad(self):
      return self.word2idx["<PAD>"]

### 重写 dataset 所需要的 '__init__', '__getitem__', '__len__'

In [10]:
'''
data.py
dataset 所需要的 '__init__', '__getitem__', '__len__'
之后配合 dataloader 使用
'''
import torch
from torch.utils.data import Dataset
class myDataset(Dataset):
    def __init__(self,X,y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label == None:
            return  self.data[idx]
        else:
            return  self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

### 模型定义

In [11]:
'''
model.py
模型定义
'''
import torch
from torch import nn
class myLSTM(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super(myLSTM,self).__init__()
        # 初始化 embedding lookup表，
        self.embed = torch.nn.Embedding(embedding.size(0),embedding.size(1)) # 0 -> 词数，1 -> 向量的大小
        # 给上纪的表赋予之前处理得到的embedding，不给weight赋值的话就是随即值
        self.embed.weight = torch.nn.Parameter(embedding)
        # 是否將 embedding fix 住，如果 fix_embedding 為 False，在訓練過程中，embedding 也會跟著被訓練
        if fix_embedding:
            self.embed.weight.requires_grad = False
        else:
            self.embed.weight.requires_grad = True
        # self.embed_dim = embedding.size(1)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.classifier = nn.Sequential(nn.Dropout(dropout),
                                        nn.Linear(hidden_dim, 1),
                                        nn.Sigmoid())
    # 定义forward流程
    def forward(self, inputs):
        inputs = self.embed(inputs)
        ##  此处不在意cn，hn所以为None
        x, _ = self.lstm(inputs, None)
        # 因为batch_first=True，所以 x 的 dimension (batch, seq_len, hidden_size)
        # 取用 LSTM 最後一層的 hidden state
        x = x[:, -1, :]
        x = self.classifier(x)
        return x

### 训练


In [13]:
'''
train.py
'''
## self-learning 把大于阀值的pred赋予label
def add_label(outputs, threshold=0.9):
    id = (outputs>=threshold) | (outputs<1-threshold)
    outputs[outputs>=threshold] = 1 # 大于等于 threshold 为正面
    outputs[outputs<1-threshold] = 0 # 小于 threshold 为负面
    return outputs.long(), id

def training(batch_size, n_epoch, lr,  X_train, y_train, train_x_no_label, train, valid, model, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    loss = nn.BCELoss() # 定義損失函數，這裡我們使用 binary cross entropy loss
    t_batch = len(train)
    v_batch = len(valid)
    optimizer = optim.Adam(model.parameters(), lr=lr)  # 將模型的參數給 optimizer，並給予適當的 learning rate
    total_loss, total_acc, best_acc = 0, 0, 0
    for epoch in range(n_epoch):
        print(X_train.shape)
        model.train()
        total_loss, total_acc = 0, 0
        # 這段做 training
        for i, (inputs, labels) in enumerate(train):
            # print("inputs_train: ", inputs)
            inputs = inputs.to(device, dtype=torch.long)  # device 為 "cuda"，將 inputs 轉成 torch.cuda.LongTensor
            labels = labels.to(device, dtype=torch.float)  # 將 labels 轉成 torch.cuda.FloatTensor，因為等等要餵進 criterion，所以型態要是 float
            optimizer.zero_grad()  # 由於 loss.backward() 的 gradient 會累加，所以每次餵完一個 batch 後需要歸零
            outputs = model(inputs)  # 將 input 餵給模型
            outputs = outputs.squeeze()  # 去掉最外面的 dimension，好讓 outputs 可以餵進 criterion(), 比如10 classes，output为(10,1),要把它转换为(10,)
            batch_loss = loss(outputs, labels)  # 計算此時模型的 training loss
            batch_loss.backward()  # 算 loss 的 gradient
            optimizer.step()  # 更新訓練模型的參數
            accuracy = evaluation(outputs, labels)  # 計算此時模型的 training accuracy
            total_acc += (accuracy / batch_size)
            total_loss += batch_loss.item()
            # print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(
            #     epoch + 1, i + 1, t_batch, loss.item(), correct * 100 / batch_size), end='\r')
        print('Epoch | {}/{}'.format(epoch+1,n_epoch))
        print('\nTrain | Loss:{:.5f} Acc: {:.3f}'.format(total_loss / t_batch, total_acc / t_batch * 100))

        # 這段做 validation
        model.eval()  # 將 model 的模式設為 eval，這樣 model 的參數就會固定住
        # self-training
        if epoch >= 4 :
          train_no_label_dataset = myDataset(X=train_x_no_label, y=None) 
          train_no_label_loader = torch.utils.data.DataLoader(
              train_no_label_dataset, 
              batch_size = batch_size, 
              shuffle = False, 
              num_workers = 0)
          with torch.no_grad():
             for i, (inputs) in enumerate(train_no_label_loader):
              #  print("inputs: ", inputs)
               inputs = inputs.to(device, dtype=torch.long) # 因为 device 为 "cuda"，将 inputs 转成 torch.cuda.LongTensor
               outputs = model(inputs) # 模型输入Input，输出output
               outputs = outputs.squeeze() # 去掉最外面的 dimension，好让 outputs 可以丢进 loss()
              #  print("outputs: ", outputs)
               labels, id = add_label(outputs)
               # 加入新标注的数据
               X_train = torch.cat((X_train.to(device), inputs[id]), dim=0)
               y_train = torch.cat((y_train.to(device), labels[id]), dim=0)
               if i == 0: 
                 train_x_no_label = inputs[~id]
               else: 
                 train_x_no_label = torch.cat((train_x_no_label.to(device), inputs[~id]), dim=0)

        # validation
        if valid is None:
            torch.save(model, "ckpt.model")
        else:
            with torch.no_grad():
                total_loss, total_acc = 0, 0
            
                for i, (inputs, labels) in enumerate(valid):
                    inputs = inputs.to(device, dtype=torch.long) # 因为 device 为 "cuda"，将 inputs 转成 torch.cuda.LongTensor
                    labels = labels.to(device, dtype=torch.float) # 因为 device 为 "cuda"，将 labels 转成 torch.cuda.FloatTensor，loss()需要float
            
                    outputs = model(inputs) # 模型输入Input，输出output
                    outputs = outputs.squeeze() # 去掉最外面的 dimension，好让 outputs 可以丢进 loss()
                    batch_loss = loss(outputs, labels) # 计算模型此时的 training loss
                    accuracy = evaluation(outputs, labels) # 计算模型此时的 training accuracy
                    total_acc += (accuracy / batch_size)
                    total_loss += batch_loss.item()

                v_batch = len(valid)
                print("Valid | Loss:{:.5f} Acc: {:.3f} ".format(total_loss/v_batch, total_acc/v_batch*100))
                if total_acc > best_acc:
                    # 如果 validation 的结果优于之前所有的結果，就把当下的模型保存下来，用于之后的testing
                    best_acc = total_acc
                    torch.save(model, "ckpt.model")
        print('-----------------------------------------------')

### 测试

In [14]:
'''
test.py
'''
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs>=0.5] = 1 # 大於等於 0.5 為正面
            outputs[outputs<0.5] = 0 # 小於 0.5 為負面
            ret_output += outputs.int().tolist()

    return ret_output

### 主程序


In [ ]:
!python -m torch.utils.collect_env

In [6]:
! nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [15]:
'''
main.py
主程序
'''
import os
import torch
import argparse
import numpy as np
import pandas as pd
from torch import nn
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn.functional as F

# 通過 torch.cuda.is_available() 的回傳值進行判斷是否有使用 GPU 的環境，如果有的話 device 就設為 "cuda"，沒有的話就設為 "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ",device)
path_prefix ='./'

# 處理好各個 data 的路徑
train_with_label = os.path.join(path_prefix, 'training_label.txt')
train_no_label = os.path.join(path_prefix, 'training_nolabel.txt')
testing_data = os.path.join(path_prefix, 'testing_data.txt')

w2v_path = os.path.join('w2v_all.model') # 處理 word to vec model 的路徑

# 定义句子长度、是否固定 embedding、batch 大小、epoch、learning rate 的值、model 的資料夾路徑等参数
sen_len = 20
fix_embedding = True # fix embedding during training
batch_size = 128
epoch = 10
lr = 0.001

model_dir = path_prefix # model directory for checkpoint model

print("loading training data ...") # 把 'training_label.txt' 跟 'training_nolabel.txt' 讀進來
train_x, y = load_training_data(train_with_label)
train_x_no_label = load_training_data(train_no_label)

# 对于input和label做预处理
# preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
# embedding = preprocess.make_embedding(load=True)
# print("train_x[0]: ",train_x[0] )
# train_x = preprocess.sentence_word2idx()
# print("train_x[0] after sentence_word2idx: ",train_x[0] )
# y = preprocess.labels_to_tensor(y)

# 对 input 跟 labels 做预处理
preprocess = Preprocess(sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x = preprocess.sentence_word2idx(train_x)
y = preprocess.labels_to_tensor(y)
train_x_no_label = preprocess.sentence_word2idx(train_x_no_label)

# import sys
# sys.exit()

# 製作一個 model 的對象
model = myLSTM(embedding, embedding_dim=250, hidden_dim=150, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device) # device為 "cuda"，model 使用 GPU 來訓練（餵進去的 inputs 也需要是 cuda tensor）

# 把 data 分為 training data 跟 validation data（將一部份 training data 拿去當作 validation data）
X_train, X_val, y_train, y_val = train_x[:180000], train_x[180000:], y[:180000], y[180000:]

# 把 data 做成 dataset 供 dataloader 取用
train_dataset = myDataset(X=X_train, y=y_train)
val_dataset =myDataset(X=X_val, y=y_val)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            num_workers = 8)
val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 8)
# 開始訓練
# def training(batch_size, n_epoch, lr,  X_train, y_train, train_x_no_label, train, valid, model, device):
training(batch_size, epoch, lr, X_train, y_train, train_x_no_label, train_loader, val_loader, model, device)



# 开始测试
print("loading testing data ...")
test_x = load_testing_data(testing_data)

# 对于input和label做预处理
preprocess = Preprocess(sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
test_x = preprocess.sentence_word2idx(train_x)
test_dataset = myDataset(X=test_x, y=None)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 8)
print('\nload model ...')
model = torch.load(os.path.join(model_dir, 'ckpt.model'))
outputs = testing(batch_size, test_loader, model, device)

# 寫到 csv 檔案供上傳 Kaggle
tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))],"label":outputs})
print("save csv ...")
tmp.to_csv(os.path.join(path_prefix, 'predict.csv'), index=False)
print("Finish Predicting")

device:  cuda
loading training data ...
['1', '+++$+++', 'are', 'wtf', '...', 'awww', 'thanks', '!']
Get embedding ...
loading word to vec model ...
get words #55777
total words: 55779

start training, parameter total:14186101, trainable:241351

torch.Size([180000, 20])
Epoch | 1/10

Train | Loss:0.49403 Acc: 75.160
Valid | Loss:0.45327 Acc: 78.463 
-----------------------------------------------
torch.Size([180000, 20])
Epoch | 2/10

Train | Loss:0.43496 Acc: 79.689
Valid | Loss:0.42911 Acc: 80.071 
-----------------------------------------------
torch.Size([180000, 20])
Epoch | 3/10

Train | Loss:0.41870 Acc: 80.615
Valid | Loss:0.42185 Acc: 80.349 
-----------------------------------------------
torch.Size([180000, 20])
Epoch | 4/10

Train | Loss:0.40580 Acc: 81.387
Valid | Loss:0.41782 Acc: 80.623 
-----------------------------------------------
torch.Size([180000, 20])
Epoch | 5/10

Train | Loss:0.39324 Acc: 81.994
Valid | Loss:0.42131 Acc: 80.319 
--------------------------------